In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split

In [2]:
pd.set_option('display.max_columns', 500, 'display.width', 1000, 'display.max_rows', 500)

In [3]:
data = pd.read_csv(r"data\main.csv")
print(data.shape)
data.head(2)

(261081, 32)


,Age,Gender,Cholesterol,BloodPressure,HeartRate,BMI,Smoker,Diabetes,Hypertension,FamilyHistory,PhysicalActivity,AlcoholConsumption,Diet,StressLevel,Ethnicity,Income,EducationLevel,Medication,ChestPainType,ECGResults,MaxHeartRate,ST_Depression,ExerciseInducedAngina,Slope,NumberOfMajorVessels,Thalassemia,PreviousHeartAttack,StrokeHistory,Residence,EmploymentStatus,MaritalStatus,Outcome
0,82,Male,107,157,93,28.7,0,1,1,0,3,4,Unhealthy,1,Asian,123643,High School,No,Asymptomatic,LV hypertrophy,150,0.38,No,Flat,3,Fixed defect,0,1,Suburban,Retired,Single,Heart Attack
1,30,Male,179,123,87,27.3,1,1,1,1,3,3,Moderate,9,Other,86850,College,Yes,Typical,ST-T abnormality,131,1.59,Yes,Upsloping,0,Reversible defect,0,1,Rural,Unemployed,Married,Heart Attack


In [4]:
data.Outcome.value_counts()

Outcome
No Heart Attack    130660
Heart Attack       130421
Name: count, dtype: int64

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261081 entries, 0 to 261080
Data columns (total 32 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Age                    261081 non-null  int64  
 1   Gender                 261081 non-null  object 
 2   Cholesterol            261081 non-null  int64  
 3   BloodPressure          261081 non-null  int64  
 4   HeartRate              261081 non-null  int64  
 5   BMI                    261081 non-null  float64
 6   Smoker                 261081 non-null  int64  
 7   Diabetes               261081 non-null  int64  
 8   Hypertension           261081 non-null  int64  
 9   FamilyHistory          261081 non-null  int64  
 10  PhysicalActivity       261081 non-null  int64  
 11  AlcoholConsumption     261081 non-null  int64  
 12  Diet                   261081 non-null  object 
 13  StressLevel            261081 non-null  int64  
 14  Ethnicity              261081 non-nu

In [6]:
data.isnull().sum().reset_index()

,index,0
0,Age,0
1,Gender,0
2,Cholesterol,0
3,BloodPressure,0
4,HeartRate,0
5,BMI,0
6,Smoker,0
7,Diabetes,0
8,Hypertension,0
9,FamilyHistory,0


In [7]:
data.describe()

,Age,Cholesterol,BloodPressure,HeartRate,BMI,Smoker,Diabetes,Hypertension,FamilyHistory,PhysicalActivity,AlcoholConsumption,StressLevel,Income,MaxHeartRate,ST_Depression,NumberOfMajorVessels,PreviousHeartAttack,StrokeHistory
count,261081.000000,261081.000000,261081.000000,261081.000000,261081.000000,261081.000000,261081.000000,261081.000000,261081.000000,261081.000000,261081.000000,261081.000000,261081.000000,261081.000000,261081.000000,261081.000000,261081.000000,261081.000000
mean,56.985759,199.477488,134.535114,89.463929,29.010486,0.498573,0.502101,0.500864,0.500017,2.999724,1.997870,4.999506,110009.215745,149.448298,2.503493,1.499902,0.496605,0.501955
std,15.863440,57.675390,25.985687,17.337375,6.345401,0.499999,0.499997,0.500000,0.500001,2.002540,1.415422,2.582601,51998.219984,28.863504,1.442993,1.118545,0.499989,0.499997
min,30.000000,100.000000,90.000000,60.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,20000.000000,100.000000,0.000000,0.000000,0.000000,0.000000
25%,43.000000,150.000000,112.000000,74.000000,23.500000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,3.000000,64859.000000,124.000000,1.260000,0.000000,0.000000,0.000000
50%,57.000000,199.000000,135.000000,89.000000,29.000000,0.000000,1.000000,1.000000,1.000000,3.000000,2.000000,5.000000,110045.000000,149.000000,2.510000,1.000000,0.000000,1.000000
75%,71.000000,249.000000,157.000000,105.000000,34.500000,1.000000,1.000000,1.000000,1.000000,5.000000,3.000000,7.000000,155040.000000,174.000000,3.750000,3.000000,1.000000,1.000000
max,84.000000,299.000000,179.000000,119.000000,40.000000,1.000000,1.000000,1.000000,1.000000,6.000000,4.000000,9.000000,199997.000000,199.000000,5.000000,3.000000,1.000000,1.000000


cat_col_num = []
for i in cat_col:
    if data[i].nunique()==2:
        cat_col_num.append(i)
print(len(cat_col_num))
print(cat_col_num)

feature_mapping_1 = {}
for i in cat_col_num:
    feature_mapping_1[i] = {}
    for p,q in enumerate(data[i].value_counts().keys()):
        feature_mapping_1[i][q] = p
print(feature_mapping_1)
        

In [8]:
feature_mapping=dict()
feature_mapping

{}

In [9]:
feature_mapping_1 = \
{'Gender': {'Male': 0, 'Female': 1}, \
 'Medication': {'Yes': 1, 'No': 0}, \
    'ExerciseInducedAngina': {'No': 0, 'Yes': 1}, \
        'Outcome': {'No Heart Attack': 0, 'Heart Attack': 1}}
feature_mapping.update(feature_mapping_1)
feature_mapping

{'Gender': {'Male': 0, 'Female': 1},
 'Medication': {'Yes': 1, 'No': 0},
 'ExerciseInducedAngina': {'No': 0, 'Yes': 1},
 'Outcome': {'No Heart Attack': 0, 'Heart Attack': 1}}

In [10]:
feature_mapping_2 = \
{'Diet': {'Unhealthy': 0, 'Moderate':0.5, 'Healthy': 1}, \
 'EducationLevel': {'High School': 0, 'College': 0.5, 'Postgraduate':1}, \
 'Slope': {'Upsloping': 0, 'Flat': 0.5, 'Downsloping':1}
}
feature_mapping.update(feature_mapping_2)
feature_mapping

{'Gender': {'Male': 0, 'Female': 1},
 'Medication': {'Yes': 1, 'No': 0},
 'ExerciseInducedAngina': {'No': 0, 'Yes': 1},
 'Outcome': {'No Heart Attack': 0, 'Heart Attack': 1},
 'Diet': {'Unhealthy': 0, 'Moderate': 0.5, 'Healthy': 1},
 'EducationLevel': {'High School': 0, 'College': 0.5, 'Postgraduate': 1},
 'Slope': {'Upsloping': 0, 'Flat': 0.5, 'Downsloping': 1}}

In [13]:
# Ethnicity, ChestPainType, ECGResults, Thalassemia, Residence, EmploymentStatus, 
print(data.MaritalStatus.value_counts())

MaritalStatus
Single      65424
Widowed     65275
Divorced    65228
Married     65154
Name: count, dtype: int64


In [14]:
for i,j in feature_mapping.items():
    data[i] = data[i].map(j)

In [15]:
num_col = list(data.describe().columns)
print(len(num_col))
print(num_col)

25
['Age', 'Gender', 'Cholesterol', 'BloodPressure', 'HeartRate', 'BMI', 'Smoker', 'Diabetes', 'Hypertension', 'FamilyHistory', 'PhysicalActivity', 'AlcoholConsumption', 'Diet', 'StressLevel', 'Income', 'EducationLevel', 'Medication', 'MaxHeartRate', 'ST_Depression', 'ExerciseInducedAngina', 'Slope', 'NumberOfMajorVessels', 'PreviousHeartAttack', 'StrokeHistory', 'Outcome']


In [16]:
cat_col = [i for i in data.columns if i not in num_col]
print(len(cat_col))
print(cat_col)

7
['Ethnicity', 'ChestPainType', 'ECGResults', 'Thalassemia', 'Residence', 'EmploymentStatus', 'MaritalStatus']


In [17]:
for i in cat_col:
    print(i, data[i].nunique())

Ethnicity 5
ChestPainType 4
ECGResults 3
Thalassemia 3
Residence 3
EmploymentStatus 3
MaritalStatus 4


In [12]:
for i in cat_col:
    if data[i].nunique()>2:
        print(data[i].value_counts())

Diet
Moderate     87524
Healthy      86962
Unhealthy    86595
Name: count, dtype: int64
Ethnicity
Asian       52520
White       52475
Black       52223
Hispanic    52019
Other       51844
Name: count, dtype: int64
EducationLevel
High School     87355
Postgraduate    86980
College         86746
Name: count, dtype: int64
ChestPainType
Asymptomatic    65695
Atypical        65260
Non-anginal     65165
Typical         64961
Name: count, dtype: int64
ECGResults
LV hypertrophy      87265
ST-T abnormality    86931
Normal              86885
Name: count, dtype: int64
Slope
Flat           87172
Downsloping    87020
Upsloping      86889
Name: count, dtype: int64
Thalassemia
Normal               87360
Fixed defect         87064
Reversible defect    86657
Name: count, dtype: int64
Residence
Suburban    87267
Rural       86982
Urban       86832
Name: count, dtype: int64
EmploymentStatus
Retired       87209
Unemployed    87170
Employed      86702
Name: count, dtype: int64
MaritalStatus
Single      654